# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ha'il,27.5219,41.6907,32.01,18,0,2.06,SA,1719355602
1,1,waitangi,-43.9535,-176.5597,11.57,84,100,4.02,NZ,1719355604
2,2,grytviken,-54.2811,-36.5092,-0.18,92,98,5.02,GS,1719355605
3,3,villa luvianos,18.9167,-100.4000,19.98,76,100,0.74,MX,1719355606
4,4,rodino,52.5000,80.2500,19.23,79,3,1.31,RU,1719355607


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# city_data_df.dtypes
no_cloudy_city_df = city_data_df[city_data_df["Cloudiness"] == 0]

# Drop any rows with null values
no_cloudy_city_df = no_cloudy_city_df.dropna()

# Display sample data
no_cloudy_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ha'il,27.5219,41.6907,32.01,18,0,2.06,SA,1719355602
6,6,adamstown,-25.0660,-130.1015,22.59,87,0,5.51,PN,1719355609
15,15,sarkand,45.4100,79.9186,21.58,44,0,3.57,KZ,1719355622
16,16,crane,31.3974,-102.3501,37.36,29,0,7.36,US,1719355623
34,34,kill devil hills,36.0307,-75.6760,29.00,61,0,5.14,US,1719355643
...,...,...,...,...,...,...,...,...,...,...
572,572,helong,42.5397,128.9972,19.19,76,0,1.89,CN,1719356293
579,579,egilsstadir,65.2653,-14.3948,10.95,62,0,5.66,IS,1719356303
582,582,trairi,-3.2778,-39.2689,25.52,83,0,4.42,BR,1719356175
585,585,amherst,41.3978,-82.2224,28.74,63,0,7.72,US,1719356309


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = no_cloudy_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ha'il,SA,27.5219,41.6907,18,
6,adamstown,PN,-25.0660,-130.1015,87,
15,sarkand,KZ,45.4100,79.9186,44,
16,crane,US,31.3974,-102.3501,29,
34,kill devil hills,US,36.0307,-75.6760,61,
...,...,...,...,...,...,...
572,helong,CN,42.5397,128.9972,76,
579,egilsstadir,IS,65.2653,-14.3948,62,
582,trairi,BR,-3.2778,-39.2689,83,
585,amherst,US,41.3978,-82.2224,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "categories": "accommodation.hotel",
    "filter":"",
    "limit":limit,
    "bias":"",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ha'il - nearest hotel: فندق كلاسيك
adamstown - nearest hotel: No hotel found
sarkand - nearest hotel: Сарканд
crane - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
kabalo - nearest hotel: No hotel found
ballston spa - nearest hotel: Medbery Inn & Spa
preobrazheniye - nearest hotel: No hotel found
idri - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
wuwei - nearest hotel: Hepingli Boutique Hotel
ludington - nearest hotel: Stearns Hotel
south sioux city - nearest hotel: South Sioux City Marriott Riverfront
al artawiyah - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
bani walid - nearest hotel: فندق الزيتونة
uberlandia - nearest hotel: Hotel Presidente
ritchie - nearest hotel: No hotel found
mar del plata - nearest hotel: Hotel 3 de Abril
port elizabeth - nearest hotel: Waterford Hotel
zhuanghe - nearest hotel: 非繁城品酒店
caldwell - nearest hotel: Best Western Plus Cald

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ha'il,SA,27.5219,41.6907,18,فندق كلاسيك
6,adamstown,PN,-25.0660,-130.1015,87,No hotel found
15,sarkand,KZ,45.4100,79.9186,44,Сарканд
16,crane,US,31.3974,-102.3501,29,No hotel found
34,kill devil hills,US,36.0307,-75.6760,61,Mariner Days Inn & Suites
...,...,...,...,...,...,...
572,helong,CN,42.5397,128.9972,76,No hotel found
579,egilsstadir,IS,65.2653,-14.3948,62,Valaskjálf
582,trairi,BR,-3.2778,-39.2689,83,Pousada Villa Aurora
585,amherst,US,41.3978,-82.2224,63,Sandstone Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols = ['Country', 'Hotel Name'],
    scale = 1,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)